## Loading the file

##### Tried to load the file with ray and modin but the process didn't work due to low memory 
##### File successfully loaded with pandas


### link to the file used for assignement: https://drive.google.com/file/d/1vBFe9U4yBcO4QyyfDk8zt786Vn_XihrH/view?usp=sharing

In [ ]:
# Load the file with Ray
# !pip install ray

import ray

ray.init() 

@ray.remote
def read_file(file_path, num_lines):
    with open(file_path, 'r') as file:
        lines = file.readlines()[:num_lines]
    return lines

file_path = '/content/drive/MyDrive/yelp-dataset/yelp_review.csv'  
num_lines = 30  

lines = ray.get(read_file.remote(file_path, num_lines)) 

# Print the lines
for line in lines:
    print(line)

ray.shutdown()  # Shutdown Ray

# task was killed due to low memory

In [ ]:
# !pip install modin[all]

# Loading the file using modin
import modin.pandas as md

modin_rv=md.read_csv("/content/drive/MyDrive/yelp-dataset/yelp_review.csv")
modin_rv1=modin_rv.head(30)
modin_rv1

In [30]:
#loading the file using pandas
import pandas as pd
separator = ','
reviews= pd.read_csv("/content/drive/MyDrive/yelp-dataset/yelp_review.csv",sep=separator)
# execution time: 1m22

In [ ]:
reviews1=reviews.head(30)
reviews1

##### Required Tasks below:

In [34]:
# validation functions for data columns
import yaml
import logging
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def clean_column_names(df):
    # Remove whitespace from column names
    df.columns = df.columns.str.strip()
    # Replace whitespace with underscores and convert to lowercase
    df.columns = df.columns.str.replace(' ', '_', regex=True)
    # Remove special characters and convert to lowercase
    df.columns = df.columns.str.replace('[^\w]', '', regex=True).str.lower()
    return df


In [35]:
# Writting a YAML file

%%writefile file.yaml
file_type: csv
dataset_name: trial1
file_name: test_data1
table_name: yielp_r
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - review_id
    - business_id
    - stars
    - text
   

Overwriting file.yaml


In [ ]:
#Running validations on data column names 

separator = ','  
data_df = reviews1

# load yaml and extract column names 
with open('file.yaml', 'r') as yaml_file:
    schema = yaml.safe_load(yaml_file)
    expected_columns = schema['columns']

#  comparing data file and yaml file 
num_columns_data = len(data_df.columns)
num_columns_expected = len(expected_columns)

column_names_data = set(data_df.columns)
column_names_expected = set(expected_columns)

if num_columns_data != num_columns_expected or column_names_data != column_names_expected:
    raise ValueError('Column validation failed. The data file does not match the expected schema.')



In [37]:
# write the file in gzipped format 
import gzip

separator = ','
data_df.to_csv('output_file.txt', sep=separator, index=False)

with open('output_file.txt', 'rb') as file_in:
    with gzip.open('output_file.txt.gz', 'wb') as file_out:
        file_out.writelines(file_in)


In [38]:
# creating a summary of the file 
import os

def summarize_file(file_path, separator='|'):
    # Count total number of rows
    def count_rows(file_path):
        with gzip.open(file_path, 'rt') as file:
            total_rows = sum(1 for _ in file)
        return total_rows

    total_rows = count_rows(file_path)

    # Count total number of columns
    def count_columns(file_path, separator):
        with gzip.open(file_path, 'rt') as file:
            header = file.readline().rstrip()
            total_columns = len(header.split(separator))
        return total_columns

    total_columns = count_columns(file_path, separator)

    # Get file size
    def get_file_size(file_path):
        size_bytes = os.path.getsize(file_path)
        
        return size_bytes
    size_bytes= get_file_size(file_path)

    # Return the summary
    summary = {
        'Total Rows': total_rows,
        'Total Columns': total_columns,
        'File Size (Bytes)': size_bytes
    }

    return summary

file_path = 'output_file.txt.gz'  # Specify the path to the gzipped file
summary = summarize_file(file_path)
print(summary)


{'Total Rows': 149, 'Total Columns': 1, 'File Size (Bytes)': 7171}
